In [1]:
import cv2
import numpy as np
theImage=cv2.imread("Lenna.jpg")
#先把彩色圖片轉成黑白的
theImageGray=cv2.cvtColor(theImage,cv2.COLOR_BGR2GRAY)
#秀出黑白圖片
cv2.imshow("theImageGray", theImageGray)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [2]:
#Ordered Dithering
#有序抖動法用的矩陣
DitherArray=np.array([[0.513,0.272,0.724,0.483,0.543,0.302,0.694,0.453],
 [0.151,0.755,0.091,0.966,0.181,0.758,0.121,0.936],
 [0.634,0.392,0.574,0.332,0.664,0.423,0.604,0.362],
 [0.060,0.875,0.211,0.815,0.030,0.906,0.241,0.845],
 [0.543,0.302,0.694,0.453,0.513,0.272,0.724,0.483],
 [0.181,0.758,0.121,0.936,0.151,0.755,0.091,0.936],
 [0.664,0.423,0.604,0.362,0.634,0.392,0.574,0.332],
 [0.030,0.906,0.241,0.845,0.060,0.875,0.211,0.815]])*255

#先複製出一個圖片用來處理(避免干擾到原圖)
#直接等於的話會有點像指標指向圖片
theDitheredImage = np.empty_like(theImageGray)
theDitheredImage[:]=theImageGray

#取出長寬
height,width=theDitheredImage.shape

'''
主要把影像切成很多8*8大小的矩陣,每8*8做一次處理,每點小於閥值(DitherArray) 就會變0(黑) 反之255(白)
'''
for i in range(0,height,8):
    for j in range(0,width,8):
        #一次比較8*8的SIZE
        theDitheredImage[i:i+8,j:j+8]=(DitherArray<theDitheredImage[i:i+8,j:j+8])*255

#秀出圖片
cv2.imshow("theDitheredImage", theDitheredImage)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()



In [3]:
#Error Diffusion

#有錯誤擴散權重的矩陣
errorArray=np.array([[0.0,    0.0,     0.0,     0.19040, 0.095230],
 [0.04762,0.095230,0.19040, 0.095230,0.04762 ],
 [0.02381,0.047620,0.095230,0.047620,0.02381]])

#先複製出一個圖片用來處理(避免干擾到原圖)
theErrorDiffusedImage = np.empty_like(theImageGray)
theErrorDiffusedImage[:]=theImageGray

#取出長寬值
height,width=theErrorDiffusedImage.shape

'''
因為如果以擴散權重矩陣的中心上方的那個點當作處理點
左,下,右 的與邊界的距離(Padding)都會是2
所以我們在四周都擴增2個單位(長寬都加4)
'''
tempArray=np.zeros((height+4,width+4))

#複製圖片
tempArray[2:height+2,2:width+2]=theErrorDiffusedImage

'''
只要處理矩陣中有實際圖片值的地方就好(2~height+2,2~width+2)
一個點一個點處理
'''
for i in range(2,height+2):
    for j in range(2,width+2):
        #先把該點舊的值存起來
        old = tempArray[i, j]
        #值超過127就變成255(白) 反之變成0(黑) 結果存成new
        new = (old // (127))*255
        #把該點的值更新
        tempArray[i,j] = new
        #計算誤差(與255或0的最短距離)
        E = old -new
        #把四周德權重值乘上誤差值 再加到四周
        tempArray[i:i+3,j-2:j+3] = tempArray[i:i+3,j-2:j+3] +E*errorArray

#把處理好的照片存出來
theErrorDiffusedImage=tempArray[2:height+2,2:width+2]
#秀出圖片
cv2.imshow("theErrorDiffusedImage", theErrorDiffusedImage)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()

In [4]:
#Dot Diffusion
#擴散順序矩陣
ClassMatrix=np.array([[204,  0,  5, 33, 51, 59, 23,118, 54, 69, 40,160,169,110,168,188],
 [  3,  6, 22, 36, 60, 50, 74,115,140, 82,147,164,171,142,220,214],
 [ 14,  7, 42, 16, 63, 52, 94, 56,133,152,158,177,179,208,222,  1],
 [ 15, 26, 43, 75, 79, 84,148, 81,139,136,166,102,217,219,226,  4],
 [ 17, 39, 72, 92,103,108,150,135,157,193,190,100,223,225,227, 13],
 [ 28,111, 99, 87,116,131,155,112,183,196,181,224,232,228, 12, 21],
 [ 47,120, 91,105,125,132,172,180,184,205,175,233,245,  8, 20, 41],
 [ 76, 65,129,137,165,145,178,194,206,170,229,244,246, 19, 24, 49],
 [ 80, 73,106,138,176,182,174,197,218,235,242,249,247, 18, 48, 68],
 [101,107,134,153,185,163,202,173,231,241,248,253, 44, 88, 70, 45],
 [123,141,149, 61,195,200,221,234,240,243,254, 38, 46, 77,104,109],
 [ 85, 96,156,130,203,215,230,250,251,252,255, 53, 62, 93, 86,117],
 [151,167,189,207,201,216,236,239, 25, 31, 34,113, 83, 95,124,114],
 [144,146,191,209,213,237,238, 29, 32, 55, 64, 97,126, 78,128,159],
 [187,192,198,212,  9, 10, 30, 35, 58, 67, 90, 71,122,127,154,161],
 [199,210,211,  2, 11, 27, 37, 57, 66, 89, 98,121,119,143,162,186]])

#擴散權重矩陣
ErrorArray=np.array([[0.38459,1,0.38459],
[1,0,1],
[0.38459,1,0.38459]])

#先複製出一個圖片用來處理(避免干擾到原圖)
theDotDiffusedImage = np.empty_like(theImageGray)
theDotDiffusedImage[:]=theImageGray

#取出照片大小
height,width=theDotDiffusedImage.shape

#創造出一個四周有寬度一邊框的擴散順序矩陣(因為把擴散權重矩陣的中心當處理點,四周會有一圈寬度一的邊框)
tempClassArray=np.zeros((16+2,16+2))
tempClassArray[1:16+1,1:16+1]=ClassMatrix

'''
一次取出16*16開始進行順序擴散
16*16中是從0~255依序處理
'''
for i in range(0,height,16):
    for j in range(0,width,16):
        #擴充取出的16*16圖片 讓四周有寬度一的邊框(因為把擴散權重矩陣的中心當處理點,四周會有一圈寬度一的邊框)
        tempArray=np.zeros((16+2,16+2))
        tempArray[1:16+1,1:16+1]=theDotDiffusedImage[i:i+16,j:j+16]

        #在順序矩陣中依序找到要處理的點
        for findNumber in range(0,256):
            #找到要處理點哦index
            result = np.where(ClassMatrix == findNumber)
            #因為找到的東西是兩個ARRAY(對應不同維度) 所以用ZIP把他們合起來
            place= list(zip(result[0], result[1]))

            #利用誤差擴散法的方法
            old = tempArray[1+place[0][0],1+place[0][1]]
            new = (old // 127)*255
            tempArray[1+place[0][0],1+place[0][1]] = new
            E = old -new
            sum=0
            '''
             和誤差擴散法不同的是 處理過的點不用擴散過去
             所以把四周權順序值小於等於自己的權重值都設為零
             最後再計算新的各點權重值
            '''
            tempErrorArray = np.empty_like(ErrorArray)
            tempErrorArray[:]=ErrorArray

            #檢差四周(共3*3)
            for k in range(-1,2):
                for m in range(-1,2):
                    if tempClassArray[1+place[0][0]+k,1+place[0][1]+m]<=tempClassArray[1+place[0][0],1+place[0][1]]:
                        tempErrorArray[k+1,m+1]=0;
                    sum += tempErrorArray[k + 1, m + 1]
            #計算新的各點權重值(權重矩陣的各點和要是1)
            if not sum==0:
                tempErrorArray = tempErrorArray / sum
            #更新該點及其附近點的值
            tempArray[1+place[0][0]-1:1+place[0][0]+2,1+place[0][1]-1:1+place[0][1]+2]= tempArray[1+place[0][0]-1:1+place[0][0]+2,1+place[0][1]-1:1+place[0][1]+2] +E*tempErrorArray
        #把處理好的16*16小區塊填回原圖
        theDotDiffusedImage[i:i+16,j:j+16]=tempArray[1:16+1,1:16+1]


#秀出圖片
cv2.imshow("theDotDiffusedImage", theDotDiffusedImage)
#等按按鈕後關掉視窗
cv2.waitKey(0)
cv2.destroyAllWindows()